## 1 Importing Library and Loading Dataset

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, FunctionTransformer, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from category_encoders import TargetEncoder

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import randint, uniform

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("/kaggle/input/engage-2-value-from-clicks-to-conversions/train_data.csv") 
test_data = pd.read_csv("/kaggle/input/engage-2-value-from-clicks-to-conversions/test_data.csv")

## 2 Exploratory Data Analysis

### 2.1 Duplicate Rows

In [ ]:
total_duplicates = df.duplicated().sum()
print(f"Total duplicate rows: {total_duplicates}")

### 2.2 Single value columns

In [ ]:
single_value_columns = [col for col in df.columns if df[col].nunique(dropna=False) == 1]
print("Number of Single Value Columns:", len(single_value_columns))


if single_value_columns:
    single_value_info = []
    for col in single_value_columns:
        constant_value = df[col].iloc[0]
        single_value_info.append({
            'Column': col,
            'Constant Value': constant_value,
            'Data Type': str(df[col].dtype)
        })
    
    single_value_df = pd.DataFrame(single_value_info)
    print("\nSingle Value Columns Table:")
    print("=" * 50)
    print(single_value_df.to_string(index=False))
else:
    print("\nNo single value columns found.")

### 2.3 Missing values

In [ ]:
# Replace values with np.nan
values_to_replace = ['not available in demo dataset', '(not set)', '(none)']

df['geoNetwork.continent'] = df['geoNetwork.continent'].replace(values_to_replace, np.nan)
df['trafficSource.medium'] = df['trafficSource.medium'].replace(values_to_replace, np.nan)

test_data['geoNetwork.continent'] = test_data['geoNetwork.continent'].replace(values_to_replace, np.nan)
test_data['trafficSource.medium'] = test_data['trafficSource.medium'].replace(values_to_replace, np.nan)

threshold = 0.6
missing_percentages = df.isnull().mean() * 100

high_missing_cols = missing_percentages[missing_percentages > (threshold * 100)]

missing_table = pd.DataFrame({
    'Column': high_missing_cols.index,
    'Missing_Percentage': high_missing_cols.values
}).round(2)

print("Columns with >60% missing values:")
print("\n" + "="*80)
print(f"{'Column Name':<60} {'Missing %':<10}")
print("="*80)
for col, pct in high_missing_cols.items():
    print(f"{col:<60} {pct:.2f}%")

### 2.4 High Cardinal Values

In [ ]:
cardinality_info = []
high_cardinality_cols = []

for col in df.select_dtypes(include=['object']).columns:
    unique_count = df[col].nunique(dropna=False)
    
    if unique_count > 200:
        cardinality_info.append({
            'Column_Name': col,
            'Unique_Count': unique_count,
            'Total_Rows': len(df),
            'Null_Count': df[col].isnull().sum(),
            'Data_Type': str(df[col].dtype)
        })
        high_cardinality_cols.append(col)

if cardinality_info:
    cardinality_df = pd.DataFrame(cardinality_info)
    print("High Cardinality Columns (>200 unique values):")
    print("=" * 80)
    print(cardinality_df.to_string(index=False))
    print("=" * 80)
    print(f"Total high cardinality columns found: {len(cardinality_info)}")
else:
    print("No columns with >200 unique values found.")

### 2.5 Univariate Analysis 

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(include=["object"])

In [ ]:
df.describe(include=["bool"])

### PurchaseValue

In [ ]:
plt.hist(df["purchaseValue"], bins=10,ec="black", log=True)
plt.xlabel("Purchase Value")
plt.ylabel("frequency")
plt.title("Histogram of PurchaseValue")
plt.show()

### Totals Bounces

- when totals.bounces is 1, purchaseValue drops to 0

In [ ]:
print(df[df['totals.bounces'] == 1]['purchaseValue'].value_counts(dropna=False))

### new_visits

In [ ]:
df['new_visits'].value_counts()

In [ ]:
print(df[df['new_visits'] == 1]['purchaseValue'].value_counts(dropna=False))

## userId, sessionId
- very high variability in the userId, sessinoId

In [ ]:
plt.hist(df["userId"], bins=10,ec="black")
plt.xlabel("ID")
plt.ylabel("userId")
plt.title("Histogram of userId")
plt.show()

plt.hist(df["sessionId"], bins=10,ec="black")
plt.xlabel("sessionId")
plt.ylabel("frequency")
plt.title("Histogram of sessionId")
plt.show()

In [ ]:
print(f"Total rows: {len(df)}")
id_cols = ['userId','sessionId']
for id_c in id_cols:
    print(f"Unique values in column: {df[id_c].nunique(dropna=False)}")

### Browser

In [ ]:
df['browser'].value_counts(normalize=True,dropna=False)

### GeoCluster

In [ ]:
df['geoCluster'].value_counts(normalize=True,dropna=False)

plt.hist(df["geoCluster"], bins=9,ec="black")
plt.xlabel("GeoCluster")
plt.ylabel("frequency")
plt.title("Histogram of GeoCluster")
plt.show()

### geoNetwork.networkDomain

In [ ]:
plt.hist(df["geoNetwork.networkDomain"], bins=5,ec="black")
plt.xlabel("geoNetwork.networkDomain")
plt.ylabel("frequency")
plt.title("Histogram of geoNetwork.networkDomain")
plt.show()

### gcIdPresent

In [ ]:
df['gclIdPresent'].value_counts(dropna=False, normalize=True)

### sessionNumber

- most of the sessions are shorter

In [ ]:
plt.hist(df["sessionNumber"], bins=100,ec="black")
plt.xlabel("sessionNumber")
plt.ylabel("frequency")
plt.title("Histogram of sessionNumber")
plt.show()

### trafficSource

In [ ]:
df['trafficSource'].value_counts(dropna=False, normalize=True)

### os

In [ ]:
df['os'].value_counts(dropna=False, normalize=True)

### trafficSource.medium

In [ ]:
df['trafficSource.medium'].value_counts(dropna=False, normalize=True)

### Location

In [ ]:
df['locationCountry'].value_counts(dropna=False, normalize=True)

In [ ]:
df['geoNetwork.continent'].value_counts(dropna=False, normalize=True)

In [ ]:
df['geoNetwork.subContinent'].value_counts(dropna=False, normalize=True)

### Page Views

In [ ]:
df['pageViews'].value_counts(normalize=True, dropna=False)

### Device Types

In [ ]:
plt.hist(df["deviceType"], bins=9,ec="black")
plt.xlabel("deviceType")
plt.ylabel("frequency")
plt.title("Histogram of deviceType")
plt.show()

### User Channel

In [ ]:
df['userChannel'].value_counts(dropna=False)

### totalHits

In [ ]:
df['totalHits'].value_counts(normalize=True)

### date

In [ ]:
df['date'].value_counts(dropna=False)

In [ ]:
ax1 = sns.displot(data=df, x="date", kde=True, bins=100, color="red", 
            facecolor="#FF0000", height=5, aspect=3.5)

### userChannel

In [ ]:
df['userChannel'].value_counts(normalize=True,dropna=False)

### totalHits

In [ ]:
df['totalHits'].value_counts(normalize=True,dropna=False)

### device.isMobile

In [ ]:
df['device.isMobile'].value_counts(normalize=True,dropna=False)

### sessionStart

In [ ]:
df['sessionStart_dt'] = pd.to_datetime(df['sessionStart'], unit='s')
test_data['sessionStart_dt'] = pd.to_datetime(test_data['sessionStart'], unit='s')


In [ ]:
# purchases by sessionStart 
df['sessionStart_dt'] = pd.to_datetime(df['sessionStart'], unit='s')
ax1 = sns.displot(data=df, x="sessionStart_dt", kde=True, bins=100, color="red", 
            facecolor="#FF0000", height=5, aspect=3.5)
df.drop(['sessionStart_dt'], axis=1, inplace=True)

### 2.6 Bivariate Analysis

In [ ]:
num_cols = df.select_dtypes(exclude=['object']).columns.tolist()

corr_matrix = df[num_cols].corr()

# Plot heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(corr_matrix, cmap='RdBu', annot=True, fmt=".2f", center=0,
            linewidths=0.5, linecolor='white', cbar_kws={"shrink": 0.75})

plt.title("Correlation Matrix  for train (Red to Blue)", fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Group by year and continent, summing purchaseValue
df['sessionStart_dt'] = pd.to_datetime(df['sessionStart'], unit='s')

df['year'] = df['sessionStart_dt'].dt.year
df['month'] = df['sessionStart_dt'].dt.month
df['day'] = df['sessionStart_dt'].dt.day
df['hour'] = df['sessionStart_dt'].dt.hour

yearly_hits = df.groupby(['year','geoNetwork.continent'])['purchaseValue'].sum()

yearly_hits.unstack().plot(
    kind='bar',
    stacked=True,
    colormap='Spectral',
    figsize=(13, 5),
    grid=False,
    alpha=0.9
)

plt.title('Purchase Value over Continent')
plt.xlabel('Year')
plt.ylabel('Purchase Value')
plt.legend(title='Continent', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


na_df = df[df['geoNetwork.continent'] == 'Americas']

value_by_year = na_df.groupby(['year', 'geoNetwork.subContinent'])['purchaseValue'].sum()

value_by_year.unstack().plot(
    kind='bar',
    stacked=True,
    colormap='Spectral',
    figsize=(13, 5),
    grid=False,
    alpha=0.9
)

plt.title('Purchase Value over American Continent')
plt.xlabel('Year')
plt.ylabel('Purchase Value')
plt.legend(title='Continent', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

ax1 = sns.displot(data=df, x="sessionStart_dt", weights="purchaseValue", kde=True, bins=100, 
                  color="red", facecolor="#FF0000", height=5, aspect=3.5)
ax1.set_axis_labels(x_var="Session Start", y_var="purchaseValue")

ax1 = sns.displot(data=df, x="hour", weights="purchaseValue", kde=True, bins=100, 
                  color="red", facecolor="#FF0000", height=5, aspect=3.5)
ax1.set_axis_labels(x_var="Hour", y_var="purchaseValue")


df.drop(['sessionStart_dt', 'year', 'month', 'day', 'hour'], axis=1, inplace=True)

In [ ]:
grouped = df.groupby("pageViews")["purchaseValue"].mean().reset_index()

sns.lineplot(data=grouped, x="pageViews", y="purchaseValue", marker="o")
plt.title("Average Purchase Value vs Page Views")
plt.xlabel("Page Views")
plt.ylabel("Average Purchase Value")
plt.show()

In [ ]:
grouped = df.groupby("sessionNumber")["purchaseValue"].mean().reset_index()

sns.lineplot(data=grouped, x="sessionNumber", y="purchaseValue", marker="o")
plt.title("Average Purchase Value vs sessionNumber")
plt.xlabel("Session Number")
plt.ylabel("Average Purchase Value")
plt.show()

In [ ]:
df['new_visits_mod'] = df['new_visits'].fillna(0)

sns.boxplot(data=df, x="new_visits_mod", y="purchaseValue")
plt.title("Purchase Value by New vs Returning Visitors")
plt.xlabel("New Visitor (1 = Yes, 0 = No)")
plt.ylabel("Purchase Value")
plt.show()


sns.relplot(
    data=df, x="pageViews", y="purchaseValue",
    col="new_visits_mod", kind="scatter", alpha=0.6
)
plt.suptitle("Page Views vs Purchase Value, by Visitor Type", y=1.05)
plt.show()

df.drop(['new_visits_mod'], inplace=True, axis=1)

## 3 Feature Engineering

In [ ]:
values_to_replace = ['not available in demo dataset', '(not set)']

df['geoNetwork.city'] = df['geoNetwork.city'].replace(values_to_replace, np.nan)
df['geoNetwork.metro'] = df['geoNetwork.metro'].replace(values_to_replace, np.nan)

test_data['geoNetwork.city'] = test_data['geoNetwork.city'].replace(values_to_replace, np.nan)
test_data['geoNetwork.metro'] = test_data['geoNetwork.metro'].replace(values_to_replace, np.nan)


## Combining countries
def combine_countries(df):
    df = df.copy()
    df['continent_country'] = df['geoNetwork.continent'] + ' - ' + df['geoNetwork.subContinent'] + '-' + df['locationCountry']
    return df.drop(['geoNetwork.continent', 'geoNetwork.subContinent', 'locationCountry'],axis=1)
df = combine_countries(df)
test_data = combine_countries(test_data)


## sessionStart
def engineer_datetime_features(X, datetime_col):
    #Extract the year,month,day,day of week, datetime and hour from Creation time stamp.
    X['Year'] = pd.to_datetime(X[datetime_col], unit='s').dt.year
    X['Month'] = pd.to_datetime(X[datetime_col], unit='s').dt.month
    X['Day'] = pd.to_datetime(X[datetime_col], unit='s').dt.day
    X['DayOfWeek'] = pd.to_datetime(X[datetime_col], unit='s').dt.strftime("%A")
    X["Hour"]=pd.to_datetime(X[datetime_col],unit="s").dt.hour
    return X

df = engineer_datetime_features(df, 'sessionStart')
test_data = engineer_datetime_features(test_data, 'sessionStart')


threshold = 0.6
high_missing_cols = df.columns[df.isnull().mean() > threshold]
df = df.loc[:, df.isnull().mean() <= threshold]
test_data = test_data.loc[:, test_data.isnull().mean() <= threshold]


# Drop single value columns
single_value_columns = [col for col in df.columns if df[col].nunique(dropna=False) == 1]
df.drop(single_value_columns, axis=1, inplace=True)
test_data.drop(single_value_columns, axis=1, inplace=True)


high_cardinality_cols = []
for col in df.select_dtypes(include=['object', 'category']).columns:
    unique_count = df[col].nunique(dropna=False)
    if unique_count>200:
        high_cardinality_cols.append(col)
df.drop(high_cardinality_cols, axis=1, inplace=True)
test_data.drop(high_cardinality_cols, axis=1, inplace=True)

## Drop Id column
# id_cols = ['userId','sessionId']
id_cols = ['sessionId']
df.drop(id_cols, axis=1, inplace=True)
test_data.drop(id_cols, axis=1, inplace=True)

df['userId'] = df['userId'].astype('object')


## 3  Preprocessing

In [ ]:

X = df.drop('purchaseValue', axis=1)
y = df['purchaseValue']

numeric_columns = X.select_dtypes(include=['number']).columns
categorical_columns = X.select_dtypes(include=['object', 'category']).columns


imputation_preprocessor = ColumnTransformer([
    ('num_imputer', SimpleImputer(strategy='mean'), numeric_columns),
    ('cat_imputer', SimpleImputer(strategy='most_frequent'), categorical_columns)
])

preprocessor = Pipeline([
    ('imputer', imputation_preprocessor),
    ('classifier', TargetEncoder()),
])


In [ ]:
y_binned = pd.cut(y, bins=7, labels=False)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.1, 
    stratify=y_binned,
    random_state=42
)

X_train = preprocessor.fit_transform(X_train,y_train)

X_val = preprocessor.transform(X_val)

test_data = preprocessor.transform(test_data)

print("Pipeline created and fitted successfully.")
print("Processed training data shape:", X.shape)
print("Processed validation data shape:", X_val.shape)
print("Processed test data shape:", test_data.shape)

## 4 Model Training

In [ ]:
models = {
    "XGBoost Regressor": XGBRegressor(random_state=42,verbose=-1),
    "Lasso Regression": Lasso(alpha=0.2),
    "Ridge Regression": Ridge(alpha=0.2),
    "Light Gradient Boosting Machine": LGBMRegressor(verbose=-1),
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(),
}
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    
    rmse = np.sqrt(mean_squared_error(y_val, preds))
    mae = mean_absolute_error(y_val, preds)
    r2 = r2_score(y_val, preds)
    
    results.append((model, rmse, mae, r2))

results_df = pd.DataFrame(results, columns=['Model', 'RMSE', 'MAE', 'R² Score'])
results_df = results_df.sort_values(by='RMSE', ascending=True).reset_index(drop=True)

display(results_df)

best_model = results_df.loc[0, 'Model']
print(f"Best performing model: {best_model}")

## 5 Hyperparameter Tuning

In [ ]:
# model =  LGBMRegressor(random_state=42, n_jobs=-1,verbose=-1,)
# params = {
#     "n_estimators": [150, 200, 300],
#     "max_depth": [None, 5, 10],
#     "num_leaves": [31, 50, 70],
#     "min_child_samples": [5, 10, 20]
# }

# search = RandomizedSearchCV(
#     estimator=model,
#     param_distributions=params,
#     n_iter=20,
#     cv=3,
#     scoring='neg_root_mean_squared_error',
#     n_jobs=-1,
#     verbose=-1,
#     random_state=42
# )
# search.fit(X_train, y_train)
# best_model = search.best_estimator_
# print("Best Parameters:", search.best_params_)

# preds = best_model.predict(X_val)
# r2 = r2_score(y_val, preds)


# print("r2 score ", r2)

- Best Parameters: {'num_leaves': 50, 'n_estimators': 300, 'min_child_samples': 5, 'max_depth': None}
- r2 score  0.3866706709560749

In [ ]:
# model = RandomForestRegressor(random_state=42, n_jobs=-1)
# params = {
#     "n_estimators": [120, 300, 500],
#     "max_depth": [None, 5, 10],
#     "min_samples_split": [2,4,5],
#     "min_samples_leaf": [1, 2,4,5]
# }

# search = RandomizedSearchCV(
#     estimator=model,
#     param_distributions=params,
#     n_iter=20,
#     cv=3,
#     scoring='neg_root_mean_squared_error',
#     n_jobs=-1,
#     verbose=-1,
#     random_state=42
# )
# search.fit(X_train, y_train)
# best_model = search.best_estimator_
# print("Best Parameters:", search.best_params_)

# preds = best_model.predict(X_val)
# r2 = r2_score(y_val, preds)


# print("r2 score ", r2)

- Best Parameters: {'n_estimators': 120, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': None}
- r2 score 0.3982068833082706



In [ ]:
# model = XGBRegressor(random_state=42, verbosity=0, n_jobs=-1)
# params = {
#     'n_estimators': [500, 400, 300],
#     'max_depth': [3, 5, 7, 11],
#     'learning_rate': [0.01, 0.1, 0.2, 0.5],
#     'subsample': [0.6, 0.8, 1.0],
#     'colsample_bytree': [0.6, 0.8, 1.0,0.6]
# }

# search = RandomizedSearchCV(
#     estimator=model,
#     param_distributions=params,
#     n_iter=30,
#     cv=3,
#     scoring='neg_root_mean_squared_error',
#     n_jobs=-1,
#     verbose=-1,
#     random_state=42
# )
# search.fit(X_train, y_train)
# best_model = search.best_estimator_
# print("Best Parameters:", search.best_params_)

# preds = best_model.predict(X_val)
# r2 = r2_score(y_val, preds)


# print("r2 score ", r2)

- Best Parameters: {'subsample': 0.8, 'n_estimators': 500, 'max_depth': 9, 'learning_rate': 0.1, 'colsample_bytree': 0.8}

- r2 score  0.40339698735560314

## 6 Best Model after tuning
## - XGboost
## - parameters  {'subsample': 0.8, 'n_estimators': 500, 'max_depth': 9, 'learning_rate': 0.1, 'colsample_bytree': 0.8}


In [ ]:
# parameters = {'subsample': 0.8, 'n_estimators': 500, 'max_depth': 6, 'learning_rate': 0.5, 'colsample_bytree': 0.6}
best_model =  XGBRegressor(max_depth=15, max_leaves=None,min_child_weight=None,missing=np.nan, monotone_constraints=None,multi_strategy=None,n_estimators=100,n_jobs=-1)
best_model.fit(X_train, y_train)

preds = best_model.predict(X_val)

rmse = np.sqrt(mean_squared_error(y_val, preds))
mae = mean_absolute_error(y_val, preds)
r2 = r2_score(y_val, preds)

print('r2 score', r2)

In [ ]:
y_pred=best_model.predict(test_data)

submission = pd.DataFrame({"id": range(0,test_data.shape[0]), "purchaseValue": y_pred})
submission.to_csv('submission.csv',index=False)